## Import

In [2]:
try :
  from google.colab import drive
  from google.colab import files
  drive.mount('/content/drive')
  PATH_GC = '/content/drive/My Drive' 
except Exception:
  print("Vous n'êtes pas sur Google Colab, vous êtes en local, assurez-vous d'avoir ajouter le Path du dossier Colab_sources")
  PATH_GC = './'#Emplacement du dossier /Colab_sources

Vous n'êtes pas sur Google Colab, vous êtes en local, assurez-vous d'avoir ajouter le Path du dossier Colab_sources


In [3]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn

In [4]:
X_train = pd.read_csv(PATH_GC+'/X_train.csv', index_col=0, sep=',')
X_train.columns.name = 'date'

Y_train = pd.read_csv(PATH_GC+'/Y_train.csv', index_col=0, sep=',')
Y_train.columns.name = 'date'

In [5]:
X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_train_reshape.columns = pd.Index(range(1,251), name='timeLag')

## Function

In [6]:
def checkOrthonormality(A): 
    
    bool = True
    D, F = A.shape   
    Error = pd.DataFrame(A.T @ A - torch.eye(F)).abs()
    
    if any(Error.unstack() > 1e-6):
        bool = False
     
    return bool



def metric_train(A, beta): 
    
    if not checkOrthonormality(A):
        return -1.0    
    X = torch.tensor(X_train_reshape.values)
    X = X.reshape(25200,250)
    X = X.float()

    B = A.clone()
    beta_clone = beta.clone()
    
    Ypred = torch.mm(X, B)
    Ypred = torch.mm(Ypred, beta_clone)        
    Ypred = Ypred.T

    Ytrue = torch.tensor(Y_train.values)

    Ytrue = torch.sum(torch.sqrt(Ytrue**2))
    Ypred = torch.sum(torch.sqrt(Ypred**2))

    meanOverlap = torch.mean(torch.sum((Ytrue * Ypred)))

    return  meanOverlap  





def loss(Y):
    return 1-metric_train(Y[:-10], Y[-10:])


In [7]:
def checkOrthonormality(A): 
    
    bool = True
    D, F = A.shape   
    Error = pd.DataFrame(A.T @ A - torch.eye(F)).abs()
    
    if any(Error.unstack() > 1e-6):
        bool = False
     
    return bool


def metric_train(A, beta): 
    
    if not checkOrthonormality(A):
        return -1.0    
    
    A = A.detach().numpy()
    beta = beta.detach().numpy()
    
    Ypred = (X_train_reshape @ A @ beta).unstack().T         
    Ytrue = Y_train
    
    Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)
    Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

    meanOverlap = (Ytrue * Ypred).sum().mean()

    return  meanOverlap


def loss_metric(A,beta):
    return torch.tensor(1-metric_train(A, beta), requires_grad = True)

## Model

In [9]:
model = model_cnn()
model = model.float()


#optimizer = torch.optim.Adam(model.parameters(), lr= 0.5)
#optimizer = torch.optim.ASGD(model.parameters(), lr = 0.5)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.5)

n_epochs = 2



In [10]:
model.train()
Loss = []
for j in range(n_epochs):
  total_loss = 0.0

  x = torch.tensor(X_train_reshape.values)
  x = x[None, None,:]
  
    
  Mat = model(x.float())
  A = Mat[:250].reshape(250,10)
  beta = Mat[250:].T
  beta = beta.reshape(10,1)

  loss =  loss_metric(A, beta)

  torch.autograd.set_detect_anomaly(True)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


    
  total_loss += loss.item()

  print("Epochs : "+str(j))
  print('Loss moyenne : '+str(total_loss/504))
  print('--'*20)
  Loss.append(total_loss/504)

  




torch.Size([2016])


Error: Canceled future for execute_request message before replies were done

## Test

In [80]:
def transform_submission_to_ypred(df_A_beta, x_test, y_test):
    """ Transform submission output (A, beta) into predicted returns S_t."""
    df_A_beta = df_A_beta.to_numpy()
    
    A = df_A_beta[:-10].reshape((250, 10))
    beta = df_A_beta[-10:].reshape(10)

    E = pd.DataFrame(A.T @ A - np.eye(10)).abs()  

    # check orthogonality of A
    if any(E.unstack() > 1e-6): 
        return None

    x_test = x_test.T
    y_test = y_test.T

    x_test = x_test[y_test.columns]

    x_test_reshape = pd.concat([x_test.shift(i+1).stack(dropna=False) for i in range(250)], 1).dropna()
    y_pred = (x_test_reshape @ A @ beta).unstack()

    return y_pred.T


def metric(df_y_true, df_y_pred):
    """ Compute metric. """
    if df_y_pred is None:  # If the y_pred has only zeroes, the metric is set to -1.
        return -1.0
    
    y_true = df_y_true.T
    y_pred = df_y_pred.T
    
    y_true = y_true.div(y_true.pow(2.0).sum(1).pow(0.5), 0)
    y_pred = y_pred.div(y_pred.pow(2.0).sum(1).pow(0.5), 0)

    mean_overlap = (y_true * y_pred).sum(1).mean()

    return mean_overlap

In [110]:
model.eval()
Loss = []
max = 0
X = torch.rand(2510)
Mat_fin = pd.DataFrame(X)
total_loss = 0.0
for k in range(0,504):
  x = torch.tensor(X_train_reshape[k:k+50].values).T.reshape(12500)

  Mat = model(x.float())

  Mat = Mat.view(-1)
  Mat = Mat.detach().numpy()
  Mat = pd.DataFrame(Mat)

  y_pred =  transform_submission_to_ypred(Mat, X_train, Y_train)
  loss = metric(Y_train, y_pred)

  if loss > max :
    Mat_fin = Mat
    max = loss


    print(loss, k)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only


0.00131576783166976
0.0030819666195665714
0.0032676283007407425
0.003330916532944631
0.0037191696704624395
0.004318045515839739
0.004373005033297223
0.006061245191600144


KeyboardInterrupt: ignored

In [111]:
print(Mat_fin)

             0
0    -0.067250
1     0.091883
2     0.088441
3    -0.075171
4    -0.035939
...        ...
2505 -0.012743
2506 -0.001464
2507  0.006261
2508  0.009548
2509  0.014681

[2510 rows x 1 columns]


In [112]:
Mat_fin.to_csv(PATH_GC+'/submission')